In [ ]:
#%% Training Loop 1 (dataloader) [not important]
j = 0
for sample in data_loader:    
    sample = sample.cuda()
    optimizer.zero_grad()   # zero the gradient buffers
    output = tan_m(net(sample[0]))
    loss1 = L(output[0:1], output[1:2], output[2:3])
    loss1.backward()

    optimizer.step()   #%%
    if j%500==0:
        print(loss1.cpu())
        # pdb.set_trace()
    j= j+1

In [ ]:
#%% Alternate Training Loop (functional) [not important ]

tdc =  test_data(temp)
j = 0
net.train()
for _ in range(1):
    sample = torch.from_numpy(tdc.get_data_Triplet())
    sample = sample.cuda()
    optimizer.zero_grad()   # zero the gradient buffers
    output = tan_m(net(sample))
    loss1 = L(output[0:1], output[1:2],output[2:3])
    loss1.backward()
    if j%2000==0:
        print(loss1.cpu())
        # pdb.set_trace()
    optimizer.step()   #%%
    j= j+1

In [ ]:
#%% SimCLR training loop (functional loop not efficient)

optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)


L= NT_Xent(batch_size=2, temperature=0.8, world_size=1)
L= L.cuda()

tdc =  test_data(temp) # temp should be one pickle file
j = 0
net.train()


for _ in range(10000):
    s1, s2 = torch.from_numpy(tdc.get_SIMCLR_data()[0]), torch.from_numpy(tdc.get_SIMCLR_data()[1]) 
    s1, s2 =  s1.cuda(), s2.cuda()
    optimizer.zero_grad()   # zero the gradient buffers
    o1, o2 = net(s1),  net(s2)
    loss1 = L(o1, o2)
    loss1.backward()
    optimizer.step() 
    if j%500==0:
        print(loss1.cpu())
        # pdb.set_trace()
    j= j+1

In [ ]:
class DummyDatasetMADA(data.Dataset):
    def __init__(self, window_size, n_channel, n_classes, n_samples):
        self.X = np.random.rand(n_samples, n_channel, window_size, 1)
        self.y = np.random.randint(n_classes, size=(n_samples,))

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.X.shape[0]

In [ ]:
#%% Test the data-loader section
s1, s2 = next(iter(data_loader))
xx = torch.cat((s1[0], s2[0]), 0)
xx.shape

In [ ]:
#%% Test the data-loader section by visualization
s1,s2= next(iter(data_loader))
s1_trx = trx(s1)
for i in range (7):
    plt.imshow(np.moveaxis((s1[0][i,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
    plt.figure()
    plt.imshow(np.moveaxis((s2[0][i,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
    plt.figure()

In [ ]:
s1 = next(iter(data_loader))

with torch.no_grad():
    net.train()
    s1 = trx(s1)
    s1= s1[0].cuda()
    opt = net(s1)
    opt0 = net(s1[0:1])
    
print(opt[1])
print(opt0)


In [ ]:
#%% some data visualization
plt.imshow(np.moveaxis((sample[0,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
plt.imshow(np.moveaxis((sample[0,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
plt.imshow(temp[7][0][0])

In [56]:
loss = InfoNCE()
loss(ot[0:1], ot[1:2], ot[2:])

tensor(1.9507, device='cuda:0')

In [40]:
#%% Analysis
cos = nn.CosineSimilarity()
pos = torch.exp(cos(ot[0:1], ot[1:2])/0.05)
negs = torch.exp(cos(ot[0:1], ot[2:])/0.05)
neg = torch.sum(negs)
denominator = pos+neg
print(pos)
print(negs)
print(denominator)
-torch.log(pos/denominator)

tensor([4.4990e+08], device='cuda:0')
tensor([4.3785e+08, 4.2815e+08, 4.2169e+08, 4.2480e+08, 4.2780e+08, 4.1624e+08],
       device='cuda:0')
tensor([3.0064e+09], device='cuda:0')


tensor([1.8995], device='cuda:0')